<a href="https://colab.research.google.com/github/Location-Artistry/GEO-DEV-NOTEBOOKS/blob/main/30_DAY_MAP_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **30 DAY MAP CHALLENGE**
## Scripts & Scrapers for Map Data
## Stay Calm and Map ON
Several automated scrips and Beautiful Soup scraping libraries to programmatically fetch data from websites.
Utilize Beautiful Soup4 and Google Libraries to search state websites


In [ ]:
!pip install search-engine-parser
!pip install "search-engine-parser[cli]"
!pip install beautifulsoup4 #- already installed with Colab
!pip install git+https://github.com/abenassi/Google-Search-API

import pandas as pd
import requests
import nest_asyncio
from bs4 import BeautifulSoup
from search_engine_parser import GoogleSearch
from googlesearch import search 
from googleapi import google

nest_asyncio.apply()

## Scrape wikipedia with Beautiful Soup for largest US cites list

In [66]:
# Scrape page contents
URL = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [67]:
# get all tables from page, and print number of tables
wikiTables = soup.find_all("table", attrs={"class": "wikitable sortable"})
len(wikiTables)

4

In [73]:
# searching through table data inspect if it is the target
table = wikiTables[0]
body = table.find_all("tr")
head = body[0]
head

<tr>
<th>2020<br/>rank
</th>
<th>City
</th>
<th>State<sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[c]</a></sup>
</th>
<th>2020<br/>census
</th>
<th>2010<br/>census
</th>
<th>Change
</th>
<th colspan="2">2020 land area
</th>
<th colspan="2">2020 population density
</th>
<th>Location
</th></tr>

#### We've got the correct table listing the 50 largest US cities by population   
Now let's load the table rows into a pandas dataframe

In [74]:
# Create the header row and strip the '\n' from the text
headings = []
for item in head.find_all("th"): # loop through all th elements
    item = (item.text).rstrip("\n")
    headings.append(item)
print(headings)

['2020rank', 'City', 'State[c]', '2020census', '2010census', 'Change', '2020 land area', '2020 population density', 'Location']


In [112]:
# get all rows except the header to load into dataframe
body_rows = body[1:] # All other items becomes the rest of the rows

In [116]:
all_rows = [] 
for row_num in range(len(body_rows)): # A row at a time
    row = [] # this will hold entries for one row
    for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        row.append(row_item.text)
    all_rows.append(row)
# print(all_rows)

In [ ]:
df = pd.DataFrame(data=all_rows,columns=headings)
df.describe()

Resulting in "AssertionError: 9 columns passed, passed data had 11 columns"
Some of the rows are showing 11 columns


In [111]:
# Ah ha! 9 heading though 11 columns due to separate columns for both sq mi and sq km!
for i, heads in enumerate(headings):
  display(f'{i} - {heads}')

'0 - 2020rank'

'1 - City'

'2 - State[c]'

'3 - 2020census'

'4 - 2010census'

...

'6 - 2020 land area'

'7 - 2020 land area km'

'8 - 2020 population density'

'9 - 2020 pop density sq km'

'10 - Location'

In [110]:
# added additional header rows to match row columns
headings.insert(7, '2020 land area km')
headings.insert(9, '2020 pop density sq km')

### Now attempting to load into dataframe with correct column numbers!

In [113]:
all_rows = [] 
for row_num in range(len(body_rows)): # A row at a time
    row = [] # this will hold entries for one row
    for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        row.append(row_item.text)
    all_rows.append(row)

In [115]:
# Success!
df = pd.DataFrame(data=all_rows,columns=headings)
df.describe()

,2020rank,City,State[c],2020census,2010census,Change,2020 land area,2020 land area km,2020 population density,2020 pop density sq km,Location
count,326,326,326,326,326,326,326,326,326,326,326
unique,326,316,46,326,322,306,296,296,315,306,326
top,306\n,Springfield\n,California\n,"143,617\n",0\n,+2.38%\n,23.5 sq mi\n,60.9 km2\n,"4,888/sq mi\n","1,887/km2\n",34°35′N 118°06′W﻿ / ﻿34.59°N 118.10°W﻿ / 34.59...
freq,1,3,75,1,2,2,3,3,3,3,1


## Previous workflow below

In [ ]:
abrv = ['AL', 'AK','AS', 'AZ', 'AR','CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'GU', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'MP','OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'VI', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

In [ ]:
stateList = ['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Minor Outlying Islands', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Northern Mariana Islands', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'U.S. Virgin Islands', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
# stateList = ['Alabama', 'Alaska', 'American Samoa', 'Arizona']

In [ ]:
for i, z in enumerate(stateList):
  print(f'State: {abrv[i]} - {z}')

In [ ]:
q = "Socrata open data"
df = pd.DataFrame(columns = ['STATE','RANK','URL'])
for x,state in enumerate(stateList):
  print(state,abrv[x])
  for z,i in enumerate(search((f'{state} {abrv[x]} {q}'), tld="com", num=3, stop=3, pause=2)):
    df.loc[len(df.index)] = [abrv[x],z,i]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171 entries, 0 to 170
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   STATE   171 non-null    object
 1   RANK    171 non-null    object
 2   URL     171 non-null    object
dtypes: object(3)
memory usage: 5.3+ KB


In [ ]:
df.head()

,STATE,RANK,URL
0,AL,0,https://www.opendatanetwork.com/
1,AL,1,https://uspto.data.commerce.gov/w/apaj-dzmw/de...
2,AL,2,https://data-algeohub.opendata.arcgis.com/
3,AK,0,https://data.muni.org/
4,AK,1,https://data-soa-dnr.opendata.arcgis.com/


In [ ]:
df.to_csv('SOCRATA-STATES.csv',encoding='utf-8')

In [ ]:
# Site with nice pre-made lists of all US states separated by commas
https://sceptermarketing.com/comma-separated-lists-of-us-states-abbreviations-select-options-etc/
# Open Data Network, Socrata compilation of states and regions!
https://www.opendatanetwork.com/
# Links of all states Open Data Portals!
http://www.harker.com/OpenData/socrata-data-portals.html